In [1]:
import snappy
import csv
import ast
import pickle
from ast import literal_eval
from itertools import combinations_with_replacement
######### braid words
braid_words_file=open('/Users/leomousseau/Desktop/slice genera part 2/census seminar github/braid_words.txt','r')
braid_words_string=braid_words_file.read()
braid_words_list=ast.literal_eval(braid_words_string)
braid_dict=dict(braid_words_list)
#### pd codes
pd_codes=[]
with open('/Users/leomousseau/Desktop/census knots concordance/pd_codes.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        pd_codes.append(row)
for k in pd_codes:
    k[1]=literal_eval(k[1])
pd_dict=dict(pd_codes)

In [2]:
def mirror(braid_word):
    return([-x for x in braid_word])
def reverse_orientation(braid_word):
    return(braid_word[::-1])
def connected_sum(w1,w2):
    n=max([abs(k) for k in w1])
    w3=[k+n if k>0 else -(-k+n) for k in w2]
    return(w1+w3)

In [3]:
import csv
import ast
from itertools import combinations_with_replacement

### ---- Load Invariants ----

def load_csv_as_dict(path, key_index=0, val_index=1, eval_value=True, cast=int):
    data = []
    with open(path, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            try:
                key = row[key_index]
                val = row[val_index]
                if eval_value:
                    val = ast.literal_eval(val)
                if cast:
                    val = cast(val)
                data.append((key, val))
            except:
                continue
    return dict(data)

# Paths
base_path = '/Users/leomousseau/Desktop/slice genera part 2/census seminar github'
group_path = base_path + '/group_A/computed results'
knot_floer_path = '/Users/leomousseau/Desktop/slice genera part 2/knot_floer.csv'
braid_path = base_path + '/braid_words.txt'

# Load braid words
with open(braid_path, 'r') as f:
    braid_words_list = ast.literal_eval(f.read())
braid_dict = dict(braid_words_list)
names = list(braid_dict.keys())

# Load invariants
s_dict = load_csv_as_dict(base_path + '/invariants/sinvariants.csv')
sign_dict = load_csv_as_dict(group_path + '/signature.csv', eval_value=True, cast=int)

# From knot floer
knot_floer = []
with open(knot_floer_path, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        knot_floer.append(row)

tau_dict = {}
eps_dict = {}
for row in knot_floer:
    name = row[0]
    try:
        tau_dict[name] = int(ast.literal_eval(row[7]))
    except:
        pass
    try:
        eps_dict[name] = int(ast.literal_eval(row[2]))
    except:
        pass

### ---- Concordance Filtering ----

variant_types = [
    ('identity', False),
    ('reverse', False),
    ('mirror', True),
    ('reverse_mirror', True)
]

def maybe_negate(x, flip):
    return -x if (x is not None and flip) else x

def get_variant_invariants(k2, flip, s_dict, sign_dict, tau_dict, eps_dict):
    return {
        's-invariant': maybe_negate(s_dict.get(k2), flip),
        'signature': maybe_negate(sign_dict.get(k2), flip),
        'tau': maybe_negate(tau_dict.get(k2), flip),
        'epsilon': maybe_negate(eps_dict.get(k2), flip)
    }

def check_invariants(k1, k2_variant, s_dict, sign_dict, tau_dict, eps_dict):
    invariants = {
        's-invariant': (s_dict.get(k1), k2_variant['s-invariant']),
        'signature': (sign_dict.get(k1), k2_variant['signature']),
        'tau': (tau_dict.get(k1), k2_variant['tau']),
        'epsilon': (eps_dict.get(k1), k2_variant['epsilon']),
    }
    for name, (a, b) in invariants.items():
        if a is not None and b is not None and a != b:
            return False, f"{name}: {a} vs {b}"
    return True, None

### ---- Main Filtering Loop ----

possibly_concordant = []
definitely_not_concordant = []

for k1, k2 in combinations_with_replacement(names, 2):
    for variant_name, flip in variant_types:
        k2_variant = get_variant_invariants(k2, flip, s_dict, sign_dict, tau_dict, eps_dict)
        ok, reason = check_invariants(k1, k2_variant, s_dict, sign_dict, tau_dict, eps_dict)
        if ok:
            possibly_concordant.append((k1, k2, variant_name))
        else:
            definitely_not_concordant.append((k1, k2, variant_name, reason))

### ---- Save Results ----

with open("possibly_concordant_variants.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Knot1", "Knot2", "Variant"])
    writer.writerows(possibly_concordant)

with open("definitely_not_concordant_variants.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Knot1", "Knot2", "Variant", "Obstruction"])
    writer.writerows(definitely_not_concordant)

print("✅ Done!")
print(f"✔️  Possibly concordant: {len(possibly_concordant)}")
print(f"❌ Definitely not concordant: {len(definitely_not_concordant)}")


✅ Done!
✔️  Possibly concordant: 50784
❌ Definitely not concordant: 3162328


let us also remove all census knots which are known to be slice and therefore concordant. 

In [4]:
slice_census=['m032', 'm222', 'm372', 's704', 's879', 'v1971', 'v2272', 'v2362', 'v2508', 'v2543', 'v2861', 'v3195', 'v3423', 'v3536', 't07281', 't07452', 't07599', 't08306', 't08617', 't10887', 't10974', 't11248', 't11418', 't11532', 't11577', 't12200', 't12587', 't12607', 't12630', 't12748', 'o9_12414', 'o9_18772', 'o9_18931', 'o9_20458', 'o9_20894', 'o9_22066', 'o9_29992', 'o9_30430', 'o9_31828', 'o9_34949', 'o9_35240', 'o9_36180', 'o9_36357', 'o9_36459', 'o9_37618', 'o9_37770', 'o9_38600', 'o9_39433', 'o9_39806', 'o9_39930', 'o9_39967', 'o9_40361', 'o9_40519', 'o9_40873', 'o9_41312', 'o9_41909', 'o9_42515', 'o9_42735', 'o9_43407', 'o9_43446']

In [5]:
len(slice_census)

60

In [6]:
filtered_concordant = [
    (name1, name2, kind) for (name1, name2, kind) in possibly_concordant
    if name1 not in slice_census and name2 not in slice_census
]

In [7]:
len(filtered_concordant)

30744

In [ ]:
import csv
import time

further_obstructed = []
possibly_concordant2 = []
c = 0

for name1, name2, kind in filtered_concordant[10000:]:
    c += 1
    start_time = time.time()

    # Log progress to terminal and file
    print(f"\n[{c}/{len(filtered_concordant)}] Processing: {name1}, {name2}, {kind}")
    with open("currently_processing.txt", "w") as f:
        f.write(f"{c}/{len(filtered_concordant)}: {name1}, {name2}, {kind}\n")

    try:
        w1 = braid_dict[name1]

        if kind == 'identity':
            w2 = braid_dict[name2]
        elif kind == 'reverse':
            w2 = reverse_orientation(braid_dict[name2])
        elif kind == 'mirror':
            w2 = mirror(braid_dict[name2])
        elif kind == 'reverse_mirror':
            w2 = reverse_orientation(mirror(braid_dict[name2]))
        else:
            print('❌ Unknown kind:', kind)
            continue

        w = connected_sum(w1, reverse_orientation(mirror(w2)))
        M = snappy.Link(braid_closure=w).exterior()
        spec = [(20, [0, 20])]

        p = 1
        max_attempts = 5
        attempt = 0
        success = False

        while p != 0 and attempt < max_attempts:
            try:
                print(f"Attempt {attempt+1}: Running HKL obstruction...")
                obstr = M.slice_obstruction_HKL(spec, verbose=True)
                p = 0
                success = True

                if isinstance(obstr, tuple):  # obstruction found
                    row = [name1, name2, kind, f"HKL obstruction {obstr}"]
                    further_obstructed.append(row)
                    print("✅ Obstructed:", obstr)
                else:
                    row = [name1, name2, kind]
                    possibly_concordant2.append(row)
                    print("➖ Not obstructed")

            except Exception as e:
                attempt += 1
                M.randomize()
                print(f"⚠️ Attempt {attempt} failed:", e)

        if not success:
            print(f"❌ Totally stuck on: {name1}, {name2}, {kind}")
            with open("stuck_pairs.csv", "a", newline='') as f:
                writer = csv.writer(f)
                writer.writerow([name1, name2, kind, "stuck after retries"])

        # Save results after each pair
        with open("further_obstructed_04_04_2025_2.csv", "w", newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(["name1", "name2", "kind", "obstruction"])
            writer.writerows(further_obstructed)

        with open("possibly_concordant_04_04_2025_2.csv", "w", newline='') as f_out:
            writer = csv.writer(f_out)
            writer.writerow(["name1", "name2", "kind"])
            writer.writerows(possibly_concordant2)

        # Optional: print how long that one took
        elapsed = time.time() - start_time
        print(f"⏱️ Time for this pair: {elapsed:.1f} sec")

    except Exception as e:
        print(f"💥 CRASH on {name1}, {name2}, {kind}: {e}")
        with open("stuck_pairs.csv", "a", newline='') as f:
            writer = csv.writer(f)
            writer.writerow([name1, name2, kind, str(e)])




   Looking at (2, 3) ...
t00027 t12395 mirror obstructed (2, 3) 1 50784
   Looking at (2, 3) ...
t00027 t12395 reverse_mirror obstructed (2, 3) 2 50784
   Looking at (2, 3) ...
   Looking at (3, 2) ...
   Looking at (3, 5) ...
   Looking at (3, 7) ...
t00027 t12757 identity obstructed (3, 7) 3 50784
   Looking at (2, 3) ...
   Looking at (3, 2) ...
   Looking at (3, 5) ...
   Looking at (3, 7) ...
t00027 t12757 reverse obstructed (3, 7) 4 50784
   Looking at (2, 3) ...
t00027 o9_00022 mirror obstructed (2, 3) 5 50784
   Looking at (2, 3) ...
t00027 o9_00022 reverse_mirror obstructed (2, 3) 6 50784


In [8]:
name1='v0321'
name2='o9_33568'
w1=braid_dict[name1]
w2=braid_dict[name2]
K1=snappy.Link(braid_closure=w1)
K2=snappy.Link(braid_closure=w2)
K1.simplify('global')
K2.simplify('global')


True

In [9]:
K1.knot_floer_homology()['tau'],K2.knot_floer_homology()['tau']

(1, -1)

In [ ]:
10034

In [6]:
len(further_obstructed),len(possibly_concordant2)

(14144, 4104)

In [51]:
########## signature function
signature_function=[]
with open('/Users/leomousseau/Desktop/slice genera part 2/census seminar github/group_A/computed results/signature_function_result.csv','r') as inp:
    reader=csv.reader(inp)
    for row in reader:
        signature_function.append(row)
del signature_function[0]
for i in range(len(signature_function)):
    l=[]
    for k in signature_function[i][1:]:
        try:
            k=literal_eval(k)
        except:
            None
        l.append(k)
    signature_function[i]=[signature_function[i][0],l]
signature_function_dict=dict(signature_function)

In [43]:
name1=possibly_concordant[30000][0]
name2=possibly_concordant[30000][1]
kind=possibly_concordant[30000][2]
name1,name2,kind

('t11497', 'o9_39339', 'mirror')

In [44]:
w1=braid_dict[name1]
w2=braid_dict[name2]
K1=snappy.Link(braid_closure=w1)
K2=snappy.Link(braid_closure=w2)

In [45]:
K1.signature(),K2.signature()

(2, -2)

In [50]:
s_dict[name1],s_dict[name2]

KeyError: 't11497'

In [47]:
tau_dict[name1],tau_dict[name2]

(-1, 1)

In [48]:
eps_dict[name1],eps_dict[name2]

(-1, 1)

In [54]:
signature_function_dict[name1],signature_function_dict[name2]

([[0.5697, 0, -1, -2, 0, 1, 0]], [[0.121, 0, 1, 2, 0, 1, 0]])

In [69]:
w=connected_sum(w1,reverse_orientation(mirror(mirror(w2))))
K=snappy.Link(braid_closure=w)
K.simplify('global')

True

In [70]:
K.signature(),K.knot_floer_homology()['tau'],K.knot_floer_homology()['epsilon'],K.knot_floer_homology()['nu']

(0, 0, 0, 0)

In [76]:
M=K.exterior()
spec=[(100,[0,100])]
obstr=M.slice_obstruction_HKL(spec,verbose=True)

   Looking at (2, 3) ...
   Looking at (3, 2) ...


In [77]:
obstr

(3, 2)

In [117]:
K=snappy.Link('6_1')

In [111]:
K=snappy.Link('4_1')

In [134]:
K=snappy.Link('3_1')

In [135]:
M=K.exterior()

In [136]:
M=K.exterior()
spec=[(100,[0,100])]
obstr=M.slice_obstruction_HKL(spec,verbose=True)
print(obstr)

   Looking at (2, 3) ...
   Looking at (3, 2) ...
None


In [128]:
K.alexander_polynomial()

t^4 - t^3 + t^2 - t + 1

In [160]:
K=snappy.Link('5_1')
M=K.exterior()
spec=[(100,[0,100])]
spec=[(10, [0, 20]), (20, [0, 10])]
obstr=M.slice_obstruction_HKL(spec,verbose=True)
print(obstr)

   Looking at (2, 5) ...
(2, 5)


In [161]:
type(obstr)==tuple

True